In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

import numpy as np

torch.manual_seed(2019)

In [0]:
import torch.nn as nn
import torch.nn.functional as F

In [0]:
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [4]:
train_data = torchvision.datasets.CIFAR100('./data/', train=True, transform= trans, download=True)
test_data = torchvision.datasets.CIFAR100('./data/', train=False, transform= trans, download=True)

0it [00:00, ?it/s]

 99%|█████████▉| 167952384/169001437 [00:16<00:00, 15989998.85it/s]

Extracting ./data/cifar-100-python.tar.gz to ./data/
Files already downloaded and verified


In [0]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64,shuffle=True)

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

import time

In [0]:
conv_channel_1 = 32
conv_channel_2 = 32
conv_channel_3 = 64
conv_channel_4 = 64
conv_channel_5 = 128

hidden1_size = 1024
#hidden2_size = 300
hidden2_size = 1024
out_size = 100
epoch = 25

In [0]:
loss = nn.CrossEntropyLoss()

In [0]:
import torch.optim as optim
from torch.autograd import Variable

def fit(epoch, data_loader, model, opt, phase):
    if phase=='train':
        volatile = False
        model.train()
    else:
        volatile = True
        model.eval()       
        
    learning_loss = 0.
    learning_correct = 0
    
    for i, (inputs, labels) in enumerate(data_loader):
        if phase == 'train':
            opt.zero_grad()
            
        inputs = Variable(inputs, volatile).cuda()
        labels = Variable(labels).cuda()
        outputs = model(inputs)
        
        loss_value = loss(outputs, labels)
        learning_loss += loss_value.item() * inputs.shape[0]
        
        if phase == 'train':
            loss_value.backward()
            opt.step()
        
        _, preds = torch.max(outputs.data, 1)
        learning_correct += (preds == labels).sum().item()
        
    epoch_loss = learning_loss /len(data_loader.dataset)
    epoch_correct = learning_correct / len(data_loader.dataset) * 100.
    
    print(f'epoch = {epoch} -> {phase:{5}} / loss = {epoch_loss:{8}.{3}}, correct = {epoch_correct:{8}.{4}}%')

In [0]:
class commonLinear(nn.Module):
    def __init__(self, nin):
        super(commonLinear, self).__init__()
        self.fc=nn.Sequential(
            nn.Linear(nin, hidden2_size),
            nn.ReLU(),
#             nn.Linear(hidden1_size, hidden2_size),
#             nn.ReLU(),
            nn.Dropout(),
            nn.Linear(hidden2_size, out_size),
            nn.ReLU()
        )
        
    def forward(self,x):
        x=self.fc(x)
        return F.log_softmax(x, dim=1)

In [0]:
class normalCNN(nn.Module):
    def __init__(self):
        super(normalCNN,self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, conv_channel_1, 3, padding=1),
            nn.BatchNorm2d(conv_channel_1),
            nn.ReLU(),
            nn.Conv2d(conv_channel_1, conv_channel_2, 3, padding=1),
            nn.BatchNorm2d(conv_channel_2),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(conv_channel_2, conv_channel_3, 3, padding=1),
            nn.BatchNorm2d(conv_channel_3),
            nn.ReLU(),
            nn.Conv2d(conv_channel_3, conv_channel_4, 3, padding=1),
            nn.BatchNorm2d(conv_channel_4),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(conv_channel_4, conv_channel_5, 3, padding=1),
            nn.BatchNorm2d(conv_channel_5),
            nn.ReLU(),
            nn.AvgPool2d(2,2)
        )
        
        self.fc = commonLinear(conv_channel_5*4*4)
        
    def forward(self,x):
        x=self.conv(x)
        x=x.view(-1,conv_channel_5*4*4)
        x=self.fc(x)
        return x

In [12]:
cnn1 = normalCNN().cuda()
opt1 = optim.SGD(cnn1.parameters(), lr=0.01, momentum=0.9)
print(cnn1)

normalCNN(
  (conv): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [15]:
for ep in range(epoch):
    t= time.time()
    fit(ep, train_loader, cnn1, opt1, 'train')
    fit(ep, test_loader, cnn1, opt1, 'eval')
    print('%.4f'%(time.time()-t))
    print()

epoch = 0 -> train / loss =    0.247, correct =    92.05%
epoch = 0 -> eval  / loss =     1.85, correct =    59.46%
35.3733

epoch = 1 -> train / loss =    0.223, correct =    92.84%
epoch = 1 -> eval  / loss =     1.95, correct =    58.94%
35.3236

epoch = 2 -> train / loss =    0.209, correct =    93.38%
epoch = 2 -> eval  / loss =     1.93, correct =    59.76%
35.4266

epoch = 3 -> train / loss =    0.187, correct =    94.14%
epoch = 3 -> eval  / loss =     1.93, correct =    59.65%
35.3508

epoch = 4 -> train / loss =    0.181, correct =    94.25%
epoch = 4 -> eval  / loss =     1.98, correct =    59.08%
35.4247

epoch = 5 -> train / loss =    0.168, correct =    94.65%
epoch = 5 -> eval  / loss =     1.99, correct =     59.5%
35.2734

epoch = 6 -> train / loss =    0.159, correct =    94.93%
epoch = 6 -> eval  / loss =     1.98, correct =    59.95%
35.3244

epoch = 7 -> train / loss =    0.147, correct =    95.29%
epoch = 7 -> eval  / loss =     2.01, correct =    59.87%
35.3180



In [0]:
class depthwiseCNN(nn.Module):
    def __init__(self,nin,nout):
        super(depthwiseCNN,self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(nin,nin,3,groups=nin, padding=1),
            nn.BatchNorm2d(nin),
            nn.ReLU(),
            nn.Conv2d(nin,nout,1),
            nn.BatchNorm2d(nout),
            nn.ReLU()
        )

    def forward(self,x):
        return self.model(x)
        
class separableCNN(nn.Module):
    def __init__(self):
        super(separableCNN,self).__init__()
        self.conv = nn.Sequential(
            depthwiseCNN(3,conv_channel_1),
            depthwiseCNN(conv_channel_1,conv_channel_2),
            nn.MaxPool2d(2,2),
            depthwiseCNN(conv_channel_2,conv_channel_3),
            depthwiseCNN(conv_channel_3,conv_channel_4),
            nn.MaxPool2d(2,2),
            depthwiseCNN(conv_channel_4,conv_channel_5),
            nn.AvgPool2d(2,2)
        )
        
        self.fc = commonLinear(conv_channel_5*4*4)
        
    def forward(self,x):
        x=self.conv(x)
        x=x.view(-1,conv_channel_5*4*4)
        x=self.fc(x)
        return x

In [0]:
cnn2 = separableCNN().cuda()
opt2 = optim.SGD(cnn2.parameters(), lr=0.01, momentum=0.9)
print(cnn2)

separableCNN(
  (conv): Sequential(
    (0): depthwiseCNN(
      (model): Sequential(
        (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=3)
        (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1))
        (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU()
      )
    )
    (1): depthwiseCNN(
      (model): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
        (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU()
      )
    )
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

In [0]:
for ep in range(epoch):
    t=time.time()
    fit(ep, train_loader, cnn2, opt2, 'train')
    fit(ep, test_loader, cnn2, opt2, 'eval')
    print('%.4f'%(time.time()-t))
    print()


epoch = 0 -> train / loss =     4.26, correct =    6.438%
epoch = 0 -> eval  / loss =     3.52, correct =    16.69%
119.0876

epoch = 1 -> train / loss =     3.38, correct =    19.11%
epoch = 1 -> eval  / loss =      2.8, correct =    29.56%
117.5487

epoch = 2 -> train / loss =     2.92, correct =    27.82%
epoch = 2 -> eval  / loss =     2.49, correct =    35.34%
117.6225

epoch = 3 -> train / loss =     2.63, correct =    33.36%
epoch = 3 -> eval  / loss =      2.4, correct =    37.95%
119.0447

epoch = 4 -> train / loss =     2.44, correct =    37.28%
epoch = 4 -> eval  / loss =     2.24, correct =    42.37%
120.0570

epoch = 5 -> train / loss =     2.28, correct =    40.43%
epoch = 5 -> eval  / loss =     2.21, correct =    42.46%
119.1794

epoch = 6 -> train / loss =     2.13, correct =    44.04%
epoch = 6 -> eval  / loss =     2.08, correct =    45.54%
118.5441

epoch = 7 -> train / loss =     2.02, correct =    46.48%
epoch = 7 -> eval  / loss =     2.09, correct =    45.09%
12

In [0]:
class inception(nn.Module):
    def __init__(self, nin, nout):
        super(inception,self).__init__()
        self.model1x1 = nn.Sequential(
            nn.Conv2d(nin, int(nout/4), 1)
        )
        self.model3x3 = nn.Sequential(
            nn.Conv2d(nin,int(nout/8),1),
            nn.Conv2d(int(nout/8),int(nout/4),3, padding=1)
        )
        self.model5x5 = nn.Sequential(
            nn.Conv2d(nin,int(nout/16),1),
            nn.Conv2d(int(nout/16),int(nout/8),3,padding=1),
            nn.Conv2d(int(nout/8),int(nout/4),3,padding=1)
        )
        self.modelmax = nn.Sequential(
            nn.MaxPool2d(3, stride=1, padding=1),
            nn.Conv2d(nin,int(nout/4),1)
        )
        
    def forward(self, x):
        part1x1=self.model1x1(x)
        part3x3=self.model3x3(x)
        part5x5=self.model5x5(x)
        partmax=self.modelmax(x)
        outputs= [part1x1, part3x3, part5x5, partmax]
        return torch.cat(outputs,1)
    
class inceptionCNN(nn.Module):
    def __init__(self):
        super(inceptionCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3,conv_channel_1, 3, padding=1),
            nn.BatchNorm2d(conv_channel_1),
            nn.ReLU(),
            nn.Conv2d(conv_channel_2, conv_channel_2, 3, padding=1),
            nn.BatchNorm2d(conv_channel_1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(conv_channel_1, conv_channel_2, 3, padding=1),
            nn.BatchNorm2d(conv_channel_2),
            nn.ReLU(),
            inception(conv_channel_2, conv_channel_3),
            nn.BatchNorm2d(conv_channel_3),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            inception(conv_channel_3, conv_channel_4),
            nn.BatchNorm2d(conv_channel_4),
            nn.ReLU(),
            inception(conv_channel_4, conv_channel_5),
            nn.BatchNorm2d(conv_channel_5),
            nn.ReLU(),
            nn.AvgPool2d(2,2)
        )
        
        self.fc=commonLinear(conv_channel_5*4*4)
    
    def forward(self,x):
        x=self.conv(x)
        x=x.view(-1, conv_channel_5*4*4)
        x=self.fc(x)
        return x

In [0]:
cnn3 = inceptionCNN().cuda()
opt3 = optim.SGD(cnn3.parameters(), lr=0.01, momentum=0.9)
print(cnn3)

inceptionCNN(
  (conv): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): inception(
      (model1x1): Sequential(
        (0): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
      )
      (model3x3): Sequential(
        (0): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
        (1): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (model5x5): Sequential(
        (0)

In [0]:
for ep in range(epoch):
    t=time.time()
    fit(ep, train_loader, cnn3, opt3, 'train')
    fit(ep, test_loader, cnn3, opt3, 'eval')
    print('%.4f'%(time.time()-t))
    print()


epoch = 0 -> train / loss =     4.18, correct =    7.584%
epoch = 0 -> eval  / loss =     3.32, correct =    20.05%
170.3735

epoch = 1 -> train / loss =     3.32, correct =    20.42%
epoch = 1 -> eval  / loss =     2.77, correct =    30.76%
170.1372

epoch = 2 -> train / loss =     2.88, correct =    28.42%
epoch = 2 -> eval  / loss =     2.57, correct =     35.4%
169.6704

epoch = 3 -> train / loss =     2.62, correct =    33.74%
epoch = 3 -> eval  / loss =     2.33, correct =    39.71%
168.1554

epoch = 4 -> train / loss =     2.42, correct =    37.74%
epoch = 4 -> eval  / loss =     2.22, correct =    41.88%
168.1624

epoch = 5 -> train / loss =     2.25, correct =     41.2%
epoch = 5 -> eval  / loss =     2.12, correct =    44.53%
167.2399

epoch = 6 -> train / loss =     2.11, correct =     44.0%
epoch = 6 -> eval  / loss =     2.15, correct =    44.45%
166.0850

epoch = 7 -> train / loss =     1.97, correct =    47.32%
epoch = 7 -> eval  / loss =     2.06, correct =    46.57%
16